In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08882737159729004
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 47.94373395608602
200000 89.756685615616
300000 109.2449305892256
400000 110.53874949539458
500000 115.60261305679549
600000 109.72469423108194
700000 110.46018169143821
800000 105.47815555772435
900000 104.74608427165575
1000000 103.53026853591633
1100000 103.78647523529689
1200000 102.6235351481435
1300000 84.03255047737835
1400000 40.97536029691503
1500000 101.42636289727356
1600000 101.37200736908885
1700000 101.71792916035373
1800000 100.8783351631166
1900000 100.46633271074732
2000000 105.52697227838073
2100000 100.88059095861536
2200000 101.52004286303345
2300000 100.83595887320203
2400000 99.78452028833563
2500000 102.279861482791
2600000 94.80396503331262
2700000 23.673623598680365
2800000 97.12809492390568
2900000 100.31178206355582
3000000 100

29300000 114.90349810654565
29400000 113.61179075134766
29500000 114.56986206323339
29600000 112.90423282928873
29700000 114.40367775771524
29800000 113.26563380298496
29900000 111.1062605259999
30000000 108.99836680627406
30100000 109.26420825927211
30200000 93.55008943301637
30300000 36.86502975146041
30400000 104.56106711488508
30500000 110.56149636763242
30600000 107.3820771137066
30700000 121.18863816681763
30800000 115.91666555737135
30900000 108.71032141140591
31000000 114.03492611510484
31100000 114.38970910147751
31200000 111.69593487737242
31300000 104.94928159054423
31400000 45.80133442217229
31500000 116.25178232152999
31600000 113.93258654016805
31700000 111.04731043062262
31800000 51.22451912979265
31900000 95.0267007329379
32000000 108.28336153899227
32100000 109.95498102786763
32200000 109.4511868941405
32300000 106.19440705537748
32400000 102.91811689859509
32500000 106.92514666522857
32600000 105.48243359421994
32700000 104.64278162779709
32800000 101.82774318345001
3

59000000 93.80632581013766
59100000 89.70906214867661
59200000 22.56559486885311
59300000 95.50933180814847
59400000 93.32600555307833
59500000 93.75635086229669
59600000 95.37979832317596
59700000 97.03095747701026
59800000 96.41105581915438
59900000 98.65794952798515
60000000 98.04702365277903
60100000 42.53684764339836
60200000 102.76345039153456
60300000 101.74662296387292
60400000 101.9410730181195
60500000 103.95639087830578
60600000 109.12028106853653
60700000 107.63560306579096
60800000 107.11120667208107
60900000 64.26441443764736
61000000 76.99780535200998
61100000 107.65605846535757
61200000 101.45827144431539
61300000 102.44752028924043
61400000 108.13048763722904
61500000 99.22929119407792
61600000 95.33098236641825
61700000 95.78140115669845
61800000 32.39539734255923
61900000 87.44685341263694
62000000 96.58576509032974
62100000 97.98350772589608
62200000 103.09891254654312
62300000 97.99865687931153
62400000 100.09225569013967
62500000 104.49593971174154
62600000 104.46

89100000 106.2151427226492
89200000 103.04789967897666
89300000 106.82252941361855
89400000 109.32748799759818
89500000 104.36601526401809
89600000 97.44663231512604
89700000 98.94878568998146
89800000 99.5307942234935
89900000 96.96740734289233
90000000 72.52461193545318
90100000 53.73606033658173
90200000 88.58575764343848
90300000 88.71380665474875
90400000 86.57305052139998
90500000 91.62351788267026
90600000 88.22049295922561
90700000 38.189060555783676
90800000 75.40669453002596
90900000 90.80473504548607
91000000 88.92249365891195
91100000 87.75228178878788
91200000 88.48266872344918
91300000 89.38648149707294
91400000 33.256612079723524
91500000 97.49275770290042
91600000 95.67735294962758
91700000 99.305444492329
91800000 96.74128801970633
91900000 99.91016131285674
92000000 107.11461912261395
92100000 104.07628744094546
92200000 101.68306362959845
92300000 100.47106339785275
92400000 100.54352495164966
92500000 99.70977616868849
92600000 99.01740720593749
92700000 29.80673397

118800000 100.81741305593391
118900000 90.70567999013265
119000000 89.02843474410184
119100000 79.85160150652992
119200000 38.728007723040726
119300000 87.83511788404213
119400000 90.69454339667413
119500000 89.06894680527546
119600000 45.22840006893115
119700000 67.60526026058753
119800000 88.19488856513098
119900000 89.4807970915245
120000000 88.68257296325692
120100000 32.821367212304196
120200000 90.82675000586462
120300000 90.21176933212644
120400000 91.69645095462066
120500000 95.22836841458881
120600000 93.73194222942881
120700000 101.8588005006781
120800000 97.20954935615508
120900000 101.68869670842133
121000000 92.24563616966752
121100000 94.03575499210677
121200000 96.21698074937778
121300000 95.89442248071741
121400000 89.16188308897549
121500000 89.74200319786969
121600000 85.22558380265221
121700000 84.59763349941325
121800000 42.90544944692844
121900000 71.8736154455924
122000000 85.32560666665648
122100000 84.38903101416129
122200000 63.46859080836438
122300000 44.88851

...

983100000 6.591005362155741
983200000 10.243165361242998
983300000 10.51774218435893
983400000 9.188729753068978
983500000 7.3446805058372675
983600000 7.333413445239467
983700000 1.8381203840216849
983800000 7.032450159978851
983900000 7.993031346987905
984000000 8.037398100350886
984100000 6.8266598021004326
984200000 7.364553975306489
984300000 3.811108011105493
984400000 3.39137358250127
984500000 7.385972870239677
984600000 6.90958821139794
984700000 7.5729206959012
984800000 7.614805917623575
984900000 6.177162174141967
985000000 3.1313318665728747
985100000 5.771712731187496
985200000 7.274412041309279
985300000 7.472658227501422
985400000 8.003950489055644
985500000 6.89952183272263
985600000 4.839519091199933
985700000 3.560082502433604
985800000 6.816646433944764
985900000 6.963096031035465
986000000 8.416090951723286
986100000 6.722801780737316
986200000 5.840122116083113
986300000 4.477304247310998
986400000 5.308455619650144
986500000 7.011977703793066
986600000 8.06500871

1011600000 2.3646969305240626
1011700000 2.69501839537647
1011800000 2.468607084694769
1011900000 1.0031135643569349
1012000000 1.2415436029352844
1012100000 1.9894279553421004
1012200000 2.8544430999023085
1012300000 2.5464806193077147
1012400000 1.7710450259740258
1012500000 2.6868929709980693
1012600000 0.5279398547563355
1012700000 1.542801806195638
1012800000 2.1257117019318965
1012900000 3.066812302491492
1013000000 2.0342921288726155
1013100000 1.5542656670070703
1013200000 1.0038151725558222
1013300000 0.6631708500211597
1013400000 1.6563198628574953
1013500000 2.726351473521101
1013600000 2.285623965269893
1013700000 1.324714794121544
1013800000 1.3288458912350118
1013900000 0.49556200112377
1014000000 1.8236056805936733
1014100000 2.0142033544442417
1014200000 1.615014308496664
1014300000 1.4146094607246233
1014400000 1.278780667704151
1014500000 0.94746963834709
1014600000 0.7934358702893151
1014700000 1.9648535221116483
1014800000 1.6902314160071739
1014900000 1.89982767067

1037300000 1.5838755894559653
1037400000 1.9345982652175313
1037500000 1.458208571234298
1037600000 1.6273583855537634
1037700000 1.7458567124898468
1037800000 1.3900712178774195
1037900000 1.678776620130196
1038000000 1.7488432943571939
1038100000 1.3661197706909003
1038200000 1.6377833501981405
1038300000 1.605104111181611
1038400000 1.4017023885795272
1038500000 1.6512499933457234
1038600000 1.9095765068043395
1038700000 1.438595201233115
1038800000 1.6744479426331647
1038900000 1.7532371141676673
1039000000 1.525683018671739
1039100000 1.5264305247199894
1039200000 1.8611197566502014
1039300000 1.7107983464266618
1039400000 1.3048516092709437
1039500000 1.8748726992586864
1039600000 1.6835337677174054
1039700000 1.1924872625619278
1039800000 1.8164217836480185
1039900000 2.099760915896183
1040000000 1.4920562685593142
1040100000 1.883002873519304
1040200000 2.2039249675433177
best so far: 0
type: [1, 3, 1, 2, 23] 138
cases of this type: 6716184
1040300000 1.452797842163525
10404000

1058400000 0.792177391040549
1058500000 1.0831385127402464
1058600000 0.898918220520356
1058700000 0.8369222385127512
1058800000 1.0228188267112432
1058900000 0.9581251337047596
1059000000 0.8328989510076087
1059100000 0.9392045738107025
1059200000 1.1260981923046733
1059300000 0.7507562901757479
1059400000 0.914375152815191
1059500000 1.1364889157152964
1059600000 0.8644176103615774
1059700000 0.732261704267484
1059800000 1.0948808746671095
1059900000 0.8967600155371295
1060000000 0.8141748936637898
1060100000 0.980094829501064
1060200000 1.0596736925652837
1060300000 0.6847625069871783
1060400000 0.8801015125320256
1060500000 1.0779860841519124
1060600000 0.8166843062107265
1060700000 0.701827032650562
1060800000 1.0297976939534075
1060900000 0.8578323370353083
1061000000 0.6694592578505139
1061100000 0.9686456201895469
1061200000 1.2732045070364415
1061300000 0.653420375858434
1061400000 0.8471814261442198
1061500000 1.006051288959396
1061600000 0.6696239951857605
1061700000 0.77483

1084800000 0.1344862145653493
1084900000 0.20462782024914025
1085000000 0.1831880961487121
1085100000 0.17211713856574498
1085200000 0.17002248635808428
1085300000 0.16951664127044477
1085400000 0.17280158709627919
1085500000 0.17247734965263803
1085600000 0.17078924849815302
1085700000 0.18225139766078194
1085800000 0.10558231003203393
1085900000 0.05943225345897409
1086000000 0.0588086923811588
1086100000 0.05850378057988683
1086200000 0.0580784110158364
1086300000 0.07493616920036542
1086400000 0.1899957731422146
1086500000 0.16175139662158355
1086600000 0.15566490842173367
1086700000 0.15422111903111935
1086800000 0.14872558014479345
1086900000 0.14636871467390855
1087000000 0.1499603989633739
1087100000 0.14685365942283735
1087200000 0.1598303387774269
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.13805553477472463
best so far: 0
type: [2, 1, 23, 1, 3] 138
cases of this type: 42849
1087400000 0.04681743655691876
best so far: 0
type: [2, 1, 23, 3